# Day 40: 70 Days Pre-Bootcamp Online Class with Data Science Nigeria - Machine Learning Stream #DSN70daysofML
Mean Shift with Titanic Dataset  <br>

In [1]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn import preprocessing

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [2]:
df = pd.read_excel('titanic.xls')
original_df = pd.DataFrame.copy(df)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
df.drop(['body', 'name'], 1, inplace=True)
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,female,29.0000,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,0.9167,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
#df.convert_objects(convert_numeric=True)
df.fillna(0, inplace=True)

In [5]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val] 
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
                    
            df[column] = list(map(convert_to_int, df[column]))
            
    return df

In [6]:
df = handle_non_numerical_data(df)
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,1,29.0000,0,0,740,211.3375,2,0,2,73
1,1,1,0,0.9167,1,2,505,151.5500,3,0,16,355
2,1,0,1,2.0000,1,2,505,151.5500,3,0,0,355
3,1,0,0,30.0000,1,2,505,151.5500,3,0,0,355
4,1,0,1,25.0000,1,2,505,151.5500,3,0,0,355


In [7]:
x = np.array(df.drop(['survived'], 1).astype(float))
x = preprocessing.scale(x)
y = np.array(df['survived'])

In [8]:
clf = MeanShift()
clf.fit(x)

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [9]:
labels = clf.labels_
clusters_centers = clf.cluster_centers_

In [10]:
original_df['cluster_group'] = np.nan

#With iloc the index of the dataframe or rows in a dataframe can be referenced
for i in range(len(x)):
    original_df['cluster_group'].iloc[i] = labels[i]

n_clusters_ = len(np.unique(labels))

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
survival_rates = {}
for i in range(n_clusters_):
    temp_df = original_df[(original_df['cluster_group'] == float(i))]
    survival_cluster = temp_df[(temp_df['survived'] == 1)]
    survival_rate = len(survival_cluster)/len(temp_df)
    survival_rates[i] = survival_rate
    
print(survival_rates)

{0: 0.3826771653543307, 1: 0.1, 2: 0.9166666666666666, 3: 0.14285714285714285}


In [12]:
print(original_df[(original_df['cluster_group'] == 1)])

      pclass  survived                                   name     sex   age  \
639        3         0            Asplund, Master. Carl Edgar    male   5.0   
640        3         0  Asplund, Master. Clarence Gustaf Hugo    male   9.0   
641        3         1      Asplund, Master. Edvin Rojj Felix    male   3.0   
642        3         0           Asplund, Master. Filip Oscar    male  13.0   
643        3         1         Asplund, Miss. Lillian Gertrud  female   5.0   
825        3         0         Goodwin, Master. Harold Victor    male   9.0   
826        3         0        Goodwin, Master. Sidney Leonard    male   1.0   
827        3         0     Goodwin, Master. William Frederick    male  11.0   
828        3         0            Goodwin, Miss. Jessie Allis  female  10.0   
829        3         0             Goodwin, Miss. Lillian Amy  female  16.0   
830        3         0            Goodwin, Mr. Charles Edward    male  14.0   
1170       3         0             Sage, Master. Tho

In [13]:
print(original_df[ (original_df['cluster_group']==1) ].describe())

       pclass   survived        age      sibsp  parch       fare  body  \
count    20.0  20.000000  12.000000  20.000000   20.0  20.000000   1.0   
mean      3.0   0.100000   9.208333   6.100000    2.0  53.214375  67.0   
std       0.0   0.307794   4.831047   1.803505    0.0  16.259352   NaN   
min       3.0   0.000000   1.000000   4.000000    2.0  31.387500  67.0   
25%       3.0   0.000000   5.000000   4.750000    2.0  43.021875  67.0   
50%       3.0   0.000000   9.500000   5.000000    2.0  46.900000  67.0   
75%       3.0   0.000000  13.250000   8.000000    2.0  69.550000  67.0   
max       3.0   1.000000  16.000000   8.000000    2.0  69.550000  67.0   

       cluster_group  
count           20.0  
mean             1.0  
std              0.0  
min              1.0  
25%              1.0  
50%              1.0  
75%              1.0  
max              1.0  


In [14]:
#The survival rate of the 1st class passengers in cluster 0, compared to the overall survival rate of cluster 0
cluster_0 = (original_df[ (original_df['cluster_group']==0) ])
cluster_0_fc = (cluster_0[ (cluster_0['pclass']==1) ])
print(cluster_0_fc.describe())

       pclass    survived         age       sibsp       parch        fare  \
count   311.0  311.000000  272.000000  311.000000  311.000000  311.000000   
mean      1.0    0.607717   39.137561    0.414791    0.321543   77.632382   
std       0.0    0.489046   14.573132    0.560794    0.642188   59.457379   
min       1.0    0.000000    0.916700    0.000000    0.000000    0.000000   
25%       1.0    0.000000   28.000000    0.000000    0.000000   30.500000   
50%       1.0    1.000000   39.000000    0.000000    0.000000   57.979200   
75%       1.0    1.000000   49.250000    1.000000    0.000000   90.539600   
max       1.0    1.000000   80.000000    3.000000    3.000000  263.000000   

             body  cluster_group  
count   35.000000          311.0  
mean   162.828571            0.0  
std     82.652172            0.0  
min     16.000000            0.0  
25%    109.500000            0.0  
50%    166.000000            0.0  
75%    233.000000            0.0  
max    307.000000         